# Experiments in connecting FAQ with legislative clauses

In [1]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd

from src.model import GoldenRetriever
from src.utils import read_txt, split_txt, make_pred, make_iscorr, make_closewrong, make_finetune, make_contrastive_finetune

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def read_and_condition_csv(csv_path, meta_col='meta', answer_col='answer', query_col='question', answer_str_col='answer', cutoff=None):
    """Only read organization meta, not personal. index=196"""
    df_pdpa = pd.read_csv(csv_path)
    if cutoff:
        df_pdpa = df_pdpa.iloc[:cutoff]
    df_pdpa['kb'] = df_pdpa[meta_col]+df_pdpa[answer_col]
    df_pdpa.rename(columns={query_col:'queries', answer_str_col:'answer_str'}, inplace=True)
    df_pdpa[answer_col] = [[x] for x in df_pdpa.index]
    df_pdpa['kb']=df_pdpa['kb'].str.replace('\n', '. ').replace('.. ', '. ')
    return df_pdpa
df_pdpa = read_and_condition_csv('../data/pdpa.csv', cutoff=196)
print(df_pdpa.columns, len(df_pdpa))
display(df_pdpa.head(3))

Index(['queries', 'meta', 'answer_str', 'kb', 'answer'], dtype='object') 196


,queries,meta,answer_str,kb,answer
0,What is personal data?,"Organisations, General\n","Personal data refers to data, whether true or ...","Organisations, General. Personal data refers t...",[0]
1,When did the PDPA come into force?,"Organisations, General\n",The PDPA was implemented in phases to allow ti...,"Organisations, General. The PDPA was implement...",[1]
2,What are the objectives of the PDPA?,"Organisations, General\n",The PDPA aims to safeguard individuals’ person...,"Organisations, General. The PDPA aims to safeg...",[2]


In [3]:
# Get train/test split index
ind = np.arange(len(df_pdpa))
np.random.seed(20)
np.random.shuffle(ind)
train_ind=ind[:]
test_ind=ind[69:]

In [4]:
# Initialize model
gr=GoldenRetriever(loss='triplet', margin=0.3)
gr.restore('../google_use_nrf_tuned/variables-0')

Instructions for updating:
Colocations handled automatically by placer.


W1106 06:29:15.681287 140063476999936 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1106 06:29:30.716362 140063476999936 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

model initiated!
Instructions for updating:
Use standard file APIs to check for files with this prefix.


W1106 06:29:59.989279 140063476999936 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from ../google_use_nrf_tuned/variables-0


I1106 06:29:59.991922 140063476999936 saver.py:1270] Restoring parameters from ../google_use_nrf_tuned/variables-0


model checkpoint restored!


## Load KB and make initial test

In [5]:
gr.load_kb(text_list=list(df_pdpa['kb']), question_list=list(df_pdpa['queries']))
df_pdpa['predictions']=df_pdpa.apply(lambda x: make_pred(x, gr), axis=1)
df_pdpa['iscorr']=df_pdpa.apply(lambda x: make_iscorr(x), axis=1)
df_pdpa.iloc[test_ind]['iscorr'].sum()/len(df_pdpa.iloc[test_ind])

knowledge base lock and loaded!


0.5669291338582677

## Finetune model

In [6]:
for epoch in range(50):
    df_pdpa['closewrong']=df_pdpa.apply(lambda x: make_closewrong(x), axis=1)
    gr.finetune(df_pdpa.iloc[train_ind]['queries'], df_pdpa.iloc[train_ind]['kb'], df_pdpa.iloc[train_ind]['kb'], list(df_pdpa.iloc[df_pdpa['closewrong']]['kb'].iloc[train_ind]), list(df_pdpa.iloc[df_pdpa['closewrong']]['kb'].iloc[train_ind]))
#     df_pdpa.iloc[train_ind].apply(lambda x: make_finetune(x, gr), axis=1)
    if epoch%5==0:
        gr.load_kb(text_list=list(df_pdpa['kb']), question_list=list(df_pdpa['queries']))
        df_pdpa['predictions']=df_pdpa.apply(lambda x: make_pred(x, gr, query_col_name='queries', top_k=2), axis=1)
        df_pdpa['iscorr']=df_pdpa.apply(lambda x: make_iscorr(x, 'predictions'), axis=1)
        print(df_pdpa.iloc[test_ind]['iscorr'].sum()/len(df_pdpa.iloc[test_ind]))
        print(df_pdpa.iloc[train_ind]['iscorr'].sum()/len(df_pdpa.iloc[train_ind]))

knowledge base lock and loaded!
0.5039370078740157
0.5306122448979592
knowledge base lock and loaded!
0.5118110236220472
0.5459183673469388
knowledge base lock and loaded!
0.5275590551181102
0.5663265306122449
knowledge base lock and loaded!
0.5748031496062992
0.6020408163265306
knowledge base lock and loaded!
0.6062992125984252
0.6377551020408163
knowledge base lock and loaded!
0.6299212598425197
0.6632653061224489
knowledge base lock and loaded!
0.6692913385826772
0.6887755102040817
knowledge base lock and loaded!
0.6771653543307087
0.6938775510204082
knowledge base lock and loaded!
0.6771653543307087
0.6989795918367347
knowledge base lock and loaded!
0.7007874015748031
0.7142857142857143


In [7]:
df_pdpa_legis = pd.read_csv('../data/pdpa_legis.csv')
df_pdpa_legis['kb'] = df_pdpa_legis['meta']+df_pdpa_legis['answer']

In [8]:
gr.load_kb(text_list=list(df_pdpa_legis['kb']), kb_name='legis_kb')
# df_pdpa['predictions_legis']=df_pdpa.apply(lambda x: make_pred(x, gr, query_col_name='prediction_str'), axis=1)

knowledge base lock and loaded!


In [11]:
df_pdpa

,queries,meta,answer_str,kb,answer,predictions,iscorr,closewrong
0,What is personal data?,"Organisations, General\n","Personal data refers to data, whether true or ...","Organisations, General. Personal data refers t...",[0],"[188, 193]",0,188
1,When did the PDPA come into force?,"Organisations, General\n",The PDPA was implemented in phases to allow ti...,"Organisations, General. The PDPA was implement...",[1],"[187, 56]",0,187
2,What are the objectives of the PDPA?,"Organisations, General\n",The PDPA aims to safeguard individuals’ person...,"Organisations, General. The PDPA aims to safeg...",[2],"[2, 134]",1,134
3,How does the PDPA benefit business?,"Organisations, General\n",The PDPA will strengthen Singapore's overall e...,"Organisations, General. The PDPA will strength...",[3],"[4, 2]",0,4
4,How will the PDPA impact business costs?,"Organisations, General\n",The provisions of the PDPA were formulated kee...,"Organisations, General. The provisions of the ...",[4],"[4, 134]",1,134
5,How is the PDPA different from the Spam Contro...,"Organisations, General\n",The Spam Control Act (SCA) sets out a framewor...,"Organisations, General. The Spam Control Act (...",[5],"[192, 5]",1,192
6,What is 'publicly available'?,"Organisations, General\n",‘Publicly available’ in relation to personal d...,"Organisations, General. ‘Publicly available’ i...",[6],"[8, 6]",1,8
7,"Can an organisation collect, use or disclose p...","Organisations, General\n",While an organisation may not obtain consent f...,"Organisations, General. While an organisation ...",[7],"[8, 7]",1,8
8,Are public databases considered publicly avail...,"Organisations, General\n",Publicly available personal data refers to per...,"Organisations, General. Publicly available per...",[8],"[8, 195]",1,195
9,Are randomly generated data considered persona...,"Organisations, General\n","Personal data is defined under the PDPA as ""da...","Organisations, General. Personal data is defin...",[9],"[10, 7]",0,10


In [ ]:
def display_ans(row):
    print('QN: ',row['queries'], '\n','CORRECT: ',row['answer_str'], '\n')
#     for pred in row['predictions']:
    for item in list(df_pdpa['kb'].iloc[row['predictions']]):
        print('FAQ: ',item)
        print('LEGIS: ', gr.make_query(item, top_k=1, predict_type='response', kb_name='legis_kb'))
#     for item in list(df_pdpa_legis['kb'].iloc[row['predictions_legis']]):
#         print('LEGIS: ', item)
    print('\n')
    
df_pdpa.apply(lambda x: display_ans(x), axis=1)

QN:  What is personal data? 
 CORRECT:  Personal data refers to data, whether true or not, about an individual who can be identified from that data; or from that data and other information to which the organisation has or is likely to have access.This includes unique identifiers (e.g. NRIC number, passport number); photographs or video images of an individual (e.g. CCTV images); as well as any set of data (e.g. name, age, address, telephone number, occupation, etc), which when taken together would be able to identify the individual. For example, Jack Lim, 36 years old, civil servant, lives at Blk 123 Bishan St 23. 

FAQ:  Individuals, General. Personal data refers to data, whether true or not, about an individual who can be identified from that data; or from that data and other information to which the organisation has or is likely to have access.This includes unique identifiers (e.g. NRIC number, passport number); as well as any set of data (e.g. name, age, address, telephone number, 

LEGIS:  (['DO NOT CALL REGISTRY, Contact information44.—(1)No person shall, on or after the prescribed date, send a specified message addressed to a Singapore telephone number unless—(a)the specified message includes clear and accurate information identifying the individual or organisation who sent or authorised the sending of the specified message;(b)the specified message includes clear and accurate information about how the recipient can readily contact that individual or organisation;(c)the specified message includes such information and complies with such conditions as is or are specified in the regulations, if any; and(d)the information included in the specified message in compliance with this subsection is reasonably likely to be valid for at least 30 days after the message is sent.(2)Any person who contravenes subsection(1) shall be guilty of an offence and shall be liable on conviction to a fine not exceeding $10,000.'], array([[0.8220055]], dtype=float32))


QN:  What is 'publ

LEGIS:  (['COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Withdrawal of consent16.—(1)On giving reasonable notice to the organisation, an individual may at any time withdraw any consent given, or deemed to have been given under this Act, in respect of the collection, use or disclosure by that organisation of personal data about the individual for any purpose.(2)On receipt of the notice referred to in subsection(1), the organisation concerned shall inform the individual of the likely consequences of withdrawing his consent.(3)An organisation shall not prohibit an individual from withdrawing his consent to the collection, use or disclosure of personal data about the individual, but this section shall not affect any legal consequences arising from such withdrawal.(4)Subject to section25, if an individual withdraws consent to the collection, use or disclosure of personal data about the individual by an organisation for any purpose, the organisation shall cease (and cause its data interme

LEGIS:  (['GENERAL, Powers of investigation50.—(1)The Commission may, upon complaint or of its own motion, conduct an investigation under this section to determine whether an organisation is not complying with this Act.(2)The powers of investigation under this section of the Commission and the inspectors shall be as set out in the Ninth Schedule.(3)The Commission may suspend, discontinue or refuse to conduct an investigation under this section if it thinks fit, including but not limited to any of the following circumstances:(a)the complainant has not complied with a direction under section27(2);(b)the parties involved in the matter have mutually agreed to settle the matter;(c)any party involved in the matter has commenced legal proceedings against another party in respect of any contravention or alleged contravention of this Act by the other party;(d)the Commission is of the opinion that the matter may be more appropriately investigated by another regulatory authority and has referred 

LEGIS:  (['COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Withdrawal of consent16.—(1)On giving reasonable notice to the organisation, an individual may at any time withdraw any consent given, or deemed to have been given under this Act, in respect of the collection, use or disclosure by that organisation of personal data about the individual for any purpose.(2)On receipt of the notice referred to in subsection(1), the organisation concerned shall inform the individual of the likely consequences of withdrawing his consent.(3)An organisation shall not prohibit an individual from withdrawing his consent to the collection, use or disclosure of personal data about the individual, but this section shall not affect any legal consequences arising from such withdrawal.(4)Subject to section25, if an individual withdraws consent to the collection, use or disclosure of personal data about the individual by an organisation for any purpose, the organisation shall cease (and cause its data interme

...

LEGIS:  (['GENERAL, Power to make regulations65.—(1)The Commission may, with the approval of the Minister, make such regulations as may be necessary or expedient for carrying out the purposes and provisions of this Act and for prescribing anything that may be required or authorised to be prescribed by this Act.[Act 22 of 2016 wef 01/10/2016](2)Without prejudice to the generality of subsection(1), the Commission may, with the approval of the Minister, make regulations for or with respect to all or any of the following matters:(a)[Deleted by Act 22 of 2016 wef 01/10/2016](b)the form, manner and procedures, relating to the making and responding to requests under section21 or 22, including the content of responses to such requests, the period for such responses, the circumstances in which an organisation may refuse to provide a response or refuse to confirm or deny the existence of any matter and the fees that an organisation may charge in respect of such requests;(c)the classes of persons

LEGIS:  (['PRELIMINARY, Purpose3.The purpose of this Act is to govern the collection, use and disclosure of personal data by organisations in a manner that recognises both the right of individuals to protect their personal data and the need of organisations to collect, use or disclose personal data for purposes that a reasonable person would consider appropriate in the circumstances.'], array([[0.5928461]], dtype=float32))
FAQ:  Organisations, Outreach and Training. The PDPC is unable to comment on courses conducted by third parties. You should assess if the curriculum offered by third parties meet your requirements before signing up for such courses. For a list of events organised by the PDPC please visit the PDPC Events page of our website. You may also consider the two-day course,  "Fundamentals of the Personal Data Protection Act (PDPA)", developed under the Business Management Workforce Skills Qualifications (BM WSQ) framework and conducted by selected Approved Training Organisati

LEGIS:  (['ACCESS TO AND CORRECTION OF PERSONAL DATA, Correction of personal data22.—(1)An individual may request an organisation to correct an error or omission in the personal data about the individual that is in the possession or under the control of the organisation.(2)Unless the organisation is satisfied on reasonable grounds that a correction should not be made, the organisation shall—(a)correct the personal data as soon as practicable; and(b)subject to subsection(3), send the corrected personal data to every other organisation to which the personal data was disclosed by the organisation within a year before the date the correction was made, unless that other organisation does not need the corrected personal data for any legal or business purpose.(3)An organisation (not being a credit bureau) may, if the individual consents, send the corrected personal data only to specific organisations to which the personal data was disclosed by the organisation within a year before the date th

LEGIS:  (['PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Functions of Commission6.The functions of the Commission shall be—(a)to promote awareness of data protection in Singapore;(b)to provide consultancy, advisory, technical, managerial or other specialist services relating to data protection;(c)to advise the Government on all matters relating to data protection;(d)to represent the Government internationally on matters relating to data protection;(e)to conduct research and studies and promote educational activities relating to data protection, including organising and conducting seminars, workshops and symposia relating thereto, and supporting other organisations conducting such activities;(f)to manage technical co-operation and exchange in the area of data protection with other organisations, including foreign data protection authorities and international or inter‑governmental organisations, on its own behalf or on behalf of the Government;(g)to administer and enforce this 

LEGIS:  (['GENERAL, Power to make regulations65.—(1)The Commission may, with the approval of the Minister, make such regulations as may be necessary or expedient for carrying out the purposes and provisions of this Act and for prescribing anything that may be required or authorised to be prescribed by this Act.[Act 22 of 2016 wef 01/10/2016](2)Without prejudice to the generality of subsection(1), the Commission may, with the approval of the Minister, make regulations for or with respect to all or any of the following matters:(a)[Deleted by Act 22 of 2016 wef 01/10/2016](b)the form, manner and procedures, relating to the making and responding to requests under section21 or 22, including the content of responses to such requests, the period for such responses, the circumstances in which an organisation may refuse to provide a response or refuse to confirm or deny the existence of any matter and the fees that an organisation may charge in respect of such requests;(c)the classes of persons

In [12]:
# gr.export(savepath='../google_use_nrf_pdpa_tuned/variables')